In [1]:
if Sys.iswindows()
    username = "Alex"
    pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
    using Pkg
    Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
else
    username = "sergey"
    pathtorepo = "/home/" *username *"/work/repo/dynamical-systems"
    using Pkg
    Pkg.activate(pathtorepo * "/env/integrate/")
end
include("/home/sergey/work/repo/dynamical-systems/system.jl")
using DifferentialEquations, DynamicalSystems, CSV, DataFrames, DelimitedFiles

  Activating project at `~/work/repo/dynamical-systems/env/integrate`


In [2]:
function traj(sys, u0, params, tspan, integ_setting)
    prob = ODEProblem(sys, u0, tspan, params)
    if integ_setting.adaptive == true
        sol = solve(prob, RK4(), adaptive = true, abstol = integ_setting.abstol, reltol = integ_setting.reltol)
    else
        sol = solve(prob, RK4(), adaptive = false, dt = integ_setting.dt, maxiters = integ_setting.maxiters)
    end

    len_sol = length(sol)
    tstart = floor(Int64, len_sol / 2)
    return sol[:, tstart:len_sol]
end

function calc_LSE(sys, u0, params, t_LSE, integ_setting)
    ds = CoupledODEs(sys, u0, params,
    diffeq = integ_setting);
    LSE = lyapunovspectrum(ds, t_LSE)
    return LSE
end

calc_LSE (generic function with 1 method)

In [3]:
sys = rate_model
params = rate_model_get_params()
params[21] = 10.0;
tspan = (0.0, 1000.0)
t_LSE = 2000
u0 = [0.3, 0.2, 0.7, 0.2, 0.001]
u0 = SVector{5}(u0)
integrator_setting = (alg = RK4(), adaptive = true, abstol = 1e-11, reltol = 1e-11)

(alg = RK4(; stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false),), adaptive = true, abstol = 1.0e-11, reltol = 1.0e-11)

In [4]:
sol = traj(sys, u0, params, tspan, integrator_setting);
LSE = calc_LSE(sys, u0, params, t_LSE, integrator_setting)

5-element Vector{Float64}:
  -0.0031011193483050804
  -0.25342190727945985
  -0.5032748593970405
  -1.7557452322756923
 -Inf

In [ ]:
sE_sv, sI_sv, rE_sv, rI_sv, Y_sv = sol[1]
sE_ev, sI_ev, rE_ev, rI_ev, Y_ev = sol[end]
l1, l2, l3, l4, l5 = LSE

In [11]:
df = DataFrame("sE_start" => sE_sv, "sI_start" => sI_sv, "rE_start" => rE_sv, "rI_start" => rI_sv, "Y_start" => Y_sv,
"sE_end" => sE_ev, "sI_end" => sI_ev, "rE_end" => rE_ev, "rI_end" => rI_ev, "Y_end" => Y_ev,
"LE1" => l1, "LE2" => l2, "LE3" => l3, "LE4" => l4, "LE5" => l5)

Row,sE_start,sI_start,rE_start,rI_start,Y_start,sE_end,sI_end,rE_end,rI_end,Y_end,LE1,LE2,LE3,LE4,LE5
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.495376,0.405511,0.183181,0.118217,2.92076e-5,0.827702,0.442497,0.440448,0.225195,0.0001,-0.00310112,-0.253422,-0.503275,-1.75575,-Inf


In [14]:
push!(df, (sE_sv, sI_sv, rE_sv, rI_sv, Y_sv, sE_ev, sI_ev, rE_ev, rI_ev, Y_ev, l1, l2, l3, l4, l5))

Row,sE_start,sI_start,rE_start,rI_start,Y_start,sE_end,sI_end,rE_end,rI_end,Y_end,LE1,LE2,LE3,LE4,LE5
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.495376,0.405511,0.183181,0.118217,2.92076e-5,0.827702,0.442497,0.440448,0.225195,0.0001,-0.00310112,-0.253422,-0.503275,-1.75575,-Inf
2,0.495376,0.405511,0.183181,0.118217,2.92076e-5,0.827702,0.442497,0.440448,0.225195,0.0001,-0.00310112,-0.253422,-0.503275,-1.75575,-Inf


In [15]:
pathtofile = "/home/sergey/work/repo/dynamical-systems/rate model/randomizer/"
namefile = "test.csv"
fullpath = pathtofile*namefile

"/home/sergey/work/repo/dynamical-systems/rate model/randomizer/test.csv"

In [16]:
CSV.write(fullpath, df)

"/home/sergey/work/repo/dynamical-systems/rate model/randomizer/test.csv"

In [ ]:
CSV.write("/home/sergey/work/repo/dynamical-systems/rate model/randomizer/output.csv",
(u0 = [[0.0, 0.0, 0.0, 0.0, 0.0]], lastpont = [[0.0, 0.0, 0.0, 0.0, 0.0]], LSE = [[0.0, 0.0, 0.0, 0.0, 0.0]]))